In [22]:
import sqlalchemy
import requests
import nest_asyncio
from requests_html import AsyncHTMLSession

In [23]:
nest_asyncio.apply()
asession = AsyncHTMLSession()

In [24]:
async def fetch_website(weburl):
    url = weburl
    r = await asession.get(url)
    
    if r.status_code == 200:
        print(f"Successfully fetched data from {url}")
    else:
        print(f"Failed to fetch data from {url}, status code: {r.status_code}")

    await r.html.arender(sleep=1)
    
    href_list = r.html.find('pre a[href]')
    href_values = [link.attrs['href'] for link in href_list]

    return href_values


In [25]:
weburl = 'https://dd.weather.gc.ca/alerts/cap/'
href_values_list = asession.loop.run_until_complete(fetch_website(weburl))
latest_date = href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/


In [26]:
print("----------------------")
print("Today's Date : ",latest_date[0:4],"-",latest_date[4:6],"-",latest_date[6:8])
print("----------------------")
new_url = weburl+latest_date+'CWTO/'
print(new_url)

----------------------
Today's Date :  2023 - 08 - 12
----------------------
https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/


In [27]:
new_href_values_list = asession.loop.run_until_complete(fetch_website(new_url))
latest_file = new_href_values_list[-1]


Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/


In [28]:
latest_file = new_href_values_list[-1]
print(latest_file)

00/


In [29]:
newz_url = new_url+latest_file
print(newz_url)

https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/00/


In [30]:
new_href_values_list = asession.loop.run_until_complete(fetch_website(newz_url))
latest_datz = new_href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/00/


In [31]:
latest_datz = new_href_values_list[-1]
print(latest_datz)

T_WHCN13_C_CWTO_202308120005_1142592915.cap


In [32]:
latest_file = newz_url+latest_datz

In [33]:
print(latest_file)

https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/00/T_WHCN13_C_CWTO_202308120005_1142592915.cap


In [34]:
req = requests.get(latest_file)

In [35]:
req.url

'https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/00/T_WHCN13_C_CWTO_202308120005_1142592915.cap'

In [36]:
import os

directory_path = r"C:\Users\supra\OneDrive\Desktop\Schulich MBAN\Sem- 1\Database fundamentals\WeatherAPI_file" # replace the file path
latest_datz = "example_file.xml" # need a loop

full_path = os.path.join(directory_path, latest_datz)

In [37]:
with open(full_path, 'wb') as f:
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

In [38]:
try:
    with open (full_path) as file:
        print(file.read())
except FileNotFoundError:
    print("I need help!!")


<?xml version='1.0' encoding='UTF-8' standalone='no'?>
<alert xmlns="urn:oasis:names:tc:emergency:cap:1.2">
    <identifier>urn:oid:2.49.0.1.124.1142592915.2023</identifier>
    <sender>cap-pac@canada.ca</sender>
    <sent>2023-08-12T00:06:43-00:00</sent>
    <status>Actual</status>
    <msgType>Update</msgType>
    <source>Env. Can. - Can. Met. Ctr. â€“ MontrÃ©al</source>
    <scope>Public</scope>
    <code>profile:CAP-CP:0.4</code>
    <code>layer:SOREM:1.0</code>
    <code>layer:EC-MSC-SMC:1.0</code>
    <code>layer:EC-MSC-SMC:1.1</code>
    <code>layer:SOREM:2.0</code>
    <note>Service Notice - February 2023: The Environment and Climate Change Canada (ECCC) CAP Service undergoes changes from time to time as the business of alerting evolves. For 2023, changes will include... 1) modifications, or even removal, of the Wireless Public Alerting Message (WPAM) layer due to the successful pilot of the UpdateX feature recently deployed in the Canadian Cell Broadcasting Environment, and 2)

In [39]:
pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


https://www.askpython.com/python-modules/xmltodict-module

In [40]:
import xmltodict

with open(full_path, 'r') as xml_file:
    xml_data = xml_file.read()


In [41]:
data_dict = xmltodict.parse(xml_data)

In [42]:
polygon_list = []

for info in data_dict['alert']['info']:
    if info.get('language') == 'en-CA':
        print("Language:", info['language'])
        print("Category:", info['category'])
        print("Event:", info['event'])
        print("Response Type:", info['responseType'])
        print("Urgency:", info['urgency'])
        print("Severity:", info['severity'])
        print("Certainty:", info['certainty'])
        print("Audience:", info['audience'])
        print("Effective:", info['effective'])
        print("Expires:", info['expires'])
        print("Sender Name:", info['senderName'])
        print("Headline:", info['headline'])
        print("Description:", info['description'])
        polygon = info['area']['polygon']
        polygon_list.append(polygon)

Language: en-CA
Category: Met
Event: squall
Response Type: Monitor
Urgency: Immediate
Severity: Severe
Certainty: Unknown
Audience: general public
Effective: 2023-08-12T00:06:43-00:00
Expires: 2023-08-12T00:05:43-00:00
Sender Name: Environment Canada
Headline: None
Description: ###


In [43]:
print("Polygon List:", polygon_list)


Polygon List: ['42.0512,-83.1462 42.0537,-83.1109 42.0629,-83.0565 42.0167,-82.9069 42.0946,-82.6165 42.2319,-82.3391 42.5988,-81.7125 42.784,-81.2109 42.66,-81.1553 42.2591,-80.9327 42.2075,-81.2457 41.6765,-82.3974 41.6765,-82.6797 41.8635,-83.0691 42.0409,-83.1496 42.0512,-83.1462']


In [44]:
polygon_str = polygon_list[0]
points_str = polygon_str.split()

polygon_points = []

for point_str in points_str:
    lat, lon = map(float, point_str.split(','))
    polygon_points.append((lat, lon))

print(polygon_points)


[(42.0512, -83.1462), (42.0537, -83.1109), (42.0629, -83.0565), (42.0167, -82.9069), (42.0946, -82.6165), (42.2319, -82.3391), (42.5988, -81.7125), (42.784, -81.2109), (42.66, -81.1553), (42.2591, -80.9327), (42.2075, -81.2457), (41.6765, -82.3974), (41.6765, -82.6797), (41.8635, -83.0691), (42.0409, -83.1496), (42.0512, -83.1462)]


In [45]:
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside


point_to_check = (42.5, -80.1)  

is_inside = point_inside_polygon(point_to_check[1], point_to_check[0], polygon_points)

if is_inside:
    print("The point is inside the polygon.")
else:
    print("The point is outside the polygon.")


The point is outside the polygon.


In [46]:
import pandas as pd

In [47]:
location_df = pd.read_csv("https://raw.githubusercontent.com/supragyabajpai/SchulichJourney_DataScience/main/DataBase/Geospatial_Coordinates.csv")

In [48]:
location_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
url = "https://www.genguide.com.ng/toronto-postal-code/"
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")

In [51]:
figure = soup.find("figure", class_="wp-block-table")

table = figure.find("table")

In [53]:
if table is not None:
    neighborhoods = []
    postal_codes = []
    districts = []

    for row in table.find_all("tr")[1:]:
        columns = row.find_all("td")
        postal_code = columns[0].text
        district = columns[1].text
        neighborhood = columns[2].text.strip()

        if district != "Not assigned":
            postal_codes.append(postal_code)
            districts.append(district)
            neighborhoods.append(neighborhood)

    data = {
        "PostalCode": postal_codes,
        "District": districts,
        "Neighborhood": neighborhoods
    }

    neighborhood_df = pd.DataFrame(data)
else:
    print("Table not found on the page.")
